In [44]:
import numpy as np
import pandas as pd
import sys
from bs4 import BeautifulSoup as BSoup
import requests

In [45]:
formLink = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247"
showLink = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247"
r = requests.get(formLink)
soup = BSoup(r.text, "lxml")
selectFields = soup.find_all("select")

In [47]:
infoField=selectFields[0].find("option", text="Informatique")["value"]
infoFieldParam=selectFields[0]["name"]
bsSem1=selectFields[2].find("option", text="Bachelor semestre 1")["value"]
bsSem6=selectFields[2].find("option", text="Bachelor semestre 6")["value"]
semParam=selectFields[2]["name"]
allYears = [y["value"] for y in selectFields[1].find_all("option")[1:]]
yearParam = selectFields[1]["name"]
htmlradiobutton=soup.find("input", type="radio")

In [51]:
def showLinkGen(htmlP, htmlV, fieldP, fieldV, yearP, yearV, semP, semV):
  return "{}&{}={}&{}={}&{}={}&{}={}".format(showLink, htmlP, htmlV, fieldP, fieldV, yearP, yearV, semP, semV)
def get_bs_html(bsSem, year):
    link = showLinkGen(htmlradiobutton["name"], htmlradiobutton["value"], infoFieldParam, infoField, yearParam, 
                 year, semParam, bsSem)
    return requests.get(link)

In [52]:
def get_bs_dataframe(request_link):
  soup2 = BSoup(request_link.text, "lxml")
  elems=soup2.find_all("tr")[2:]
  titleinfo = soup2.find("font").text.split(', ')
  semester=titleinfo[2]
  semester=int(semester[(len(semester)-2):])
  year=titleinfo[1]
  all_data=[]
  for elem in elems:
    items=elem.find_all("td")
    gender = "M" if (items[0].text == "Monsieur") else "F"
    sciper = int(items[10].text)
    all_data.append({"Scipper": sciper, "Sex": gender, "Year": year, "Semester": semester})
  return pd.DataFrame(all_data)


In [53]:
def get_bs_alldata():
  all_data = []
  for bsSem in [bsSem1, bsSem6]:
    for year in allYears:
        all_data.append(get_bs_dataframe(get_bs_html(bsSem, year)))
  return pd.concat(all_data)
get_bs_alldata()

,Scipper,Semester,Sex,Year
0,235688,1,M,2016-2017
1,274015,1,M,2016-2017
2,268410,1,F,2016-2017
3,271464,1,M,2016-2017
4,274518,1,M,2016-2017
5,249613,1,M,2016-2017
6,262214,1,M,2016-2017
7,262239,1,M,2016-2017
8,257916,1,M,2016-2017
9,271508,1,F,2016-2017


In [63]:
msSem1_text="Master semestre 1"
msSemPF_text="Projet Master automne"
msSemPS_text="Projet Master printemps"
msSem1=selectFields[2].find("option", text=msSem1_text)["value"]
msSemPF=selectFields[2].find("option", text=msSemPF_text)["value"]
msSemPS=selectFields[2].find("option", text=msSemPS_text)["value"]
r2 = get_bs_html(msSemPF, allYears[0])

In [64]:
r2.content

'<html><head><META http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><link rel="stylesheet" type="text/css" href="gedpublicreports.css?ww_x_path=Gestac.Moniteur.Style"></head><body bgcolor="#ffffff" marginheight="0" marginwidth="5" link="#666666" vlink="#666666" alink="#666666"><fieldset style="text-align:right; width:40%; position:relative; margin-right: 10px;float:right; border: 0; padding: 0 0 8px 0;"><a style="color:#990033;" href="!GEDREPORTS.html?ww_x_GPS=-1&amp;ww_i_reportModel=133685247&amp;ww_i_reportModelXsl=133685270&amp;ww_x_UNITE_ACAD=249847&amp;ww_x_PERIODE_ACAD=355925344&amp;ww_x_PERIODE_PEDAGO=249127">Identification pour acc\xe9der aux e-mails<br>Login to access email adresses</a></fieldset><script>\n  function mailList(x) {\n   var vtop = (screen.height-200)/2;\n   var vleft=(screen.width-600)/2;\n   var w=open("", "emaillist", "Scrollbars=1,resizable=1,width=600,height=200,top="+vtop+",left="+vleft);\n   w.document.write(x);\n   w.document.close();\n  

In [95]:
def get_ms_dataframe(request_link):
  soup2 = BSoup(request_link.text, "lxml")
  elems=soup2.find_all("tr")[2:]
  titleinfo = soup2.find("font")
  if(titleinfo is None):
      return pd.DataFrame([])
  else:
      titleinfo = titleinfo.text.split(', ')
      semester=titleinfo[2]
      semester=1 if semester==msSem1_text else (2 if semester==msSemPF_text else 3)
      year=titleinfo[1]
      all_data=[]
      for elem in elems:
        items=elem.find_all("td")
        gender = "M" if (items[0].text == "Monsieur") else "F"
        sciper = int(items[10].text)
        spec = items[4].text <> ""
        minor = items[6].text <> ""
        all_data.append({"Scipper": sciper, "Sex": gender, "Year": year, "Semester": semester,
                        "Minor": minor, "Specialization": spec})
      return pd.DataFrame(all_data)
get_ms_dataframe(r2)

,Minor,Scipper,Semester,Sex,Specialization,Year
0,False,166075,2,M,False,2016-2017
1,False,243163,2,M,False,2016-2017
2,False,194182,2,M,True,2016-2017
3,True,213664,2,M,False,2016-2017
4,False,214637,2,M,False,2016-2017
5,False,243164,2,M,False,2016-2017
6,False,203947,2,M,False,2016-2017
7,True,203945,2,M,False,2016-2017
8,True,244136,2,M,False,2016-2017
9,False,229467,2,M,False,2016-2017


In [96]:
def get_ms_alldata():
  all_data = []
  for sem in [msSem1, msSemPF, msSemPS]:
    for year in allYears:
        all_data.append(get_ms_dataframe(get_bs_html(sem, year)))
  return pd.concat(all_data)
data=get_ms_alldata()

In [97]:
data

,Minor,Scipper,Semester,Sex,Specialization,Year
0,False,271331,1,M,False,2016-2017
1,False,273101,1,M,False,2016-2017
2,False,267543,1,M,False,2016-2017
3,False,202293,1,M,False,2016-2017
4,False,228219,1,M,True,2016-2017
5,False,212464,1,M,False,2016-2017
6,False,226371,1,M,True,2016-2017
7,False,273107,1,M,True,2016-2017
8,False,266983,1,M,False,2016-2017
9,False,273118,1,M,False,2016-2017
